In [1]:
import os 
import pandas as pd

In [3]:
data_path = "dataset/enron1"

In [4]:
ham = []
spam = []

In [6]:
for root, dirs, files in os.walk(data_path):
    #in subdirectory
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='latin1') as f:
            content = f.read()
            #if in ham, append ham 
            if 'ham' in root:
                ham.append({'filename': file, 'content': content, 'label': 'ham'})
            #if in spam, append spam
            elif 'spam' in root:
                spam.append({'filename': file, 'content': content, 'label': 'spam'})


In [18]:
ham_df = pd.DataFrame(ham)
spam_df = pd.DataFrame(spam)
df = pd.concat([ham_df, spam_df], ignore_index = True)

In [19]:
print(df.head)

<bound method NDFrame.head of                             filename  \
0     1365.2000-06-16.farmer.ham.txt   
1     3560.2001-02-09.farmer.ham.txt   
2     0877.2000-04-06.farmer.ham.txt   
3     2937.2000-11-27.farmer.ham.txt   
4     1270.2000-06-07.farmer.ham.txt   
...                              ...   
5167     1505.2004-07-09.GP.spam.txt   
5168     2148.2004-09-13.GP.spam.txt   
5169     2406.2004-10-06.GP.spam.txt   
5170     1459.2004-06-30.GP.spam.txt   
5171     2030.2004-08-30.GP.spam.txt   

                                                content label  
0     Subject: revised sea robin availabilities effe...   ham  
1     Subject: re : january spot purchases - deals n...   ham  
2     Subject: re : buyback / deficiency deals works...   ham  
3     Subject: king ranch processed volumes at tailg...   ham  
4     Subject: confirming requisitions\nconfirming t...   ham  
...                                                 ...   ...  
5167  Subject: what she doesnt know sprig